<a href="https://colab.research.google.com/github/YusukeTakahashi2001/Pytorch-Tutorial_Des2020/blob/main/ex_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ERROR

In [2]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab Notebooks/IPhone-Macbook

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/IPhone-Macbook


In [3]:
!pip install comet_ml
from comet_ml import Experiment

     |████████████████████████████████| 245kB 7.5MB/s 
     |████████████████████████████████| 512kB 12.1MB/s 
     |████████████████████████████████| 204kB 15.5MB/s 
  Created wheel for configobj: filename=configobj-5.0.6-cp36-none-any.whl size=34547 sha256=dc14882e4861ced5775d8c9644a5cdf4118d762baeb7334b95a3a9f5de41101d
  Stored in directory: /root/.cache/pip/wheels/f1/e4/16/4981ca97c2d65106b49861e0b35e2660695be7219a2d351ee0
Successfully built configobj


In [4]:
# パッケージのimport
import numpy as np
import random

import torch
import torch.nn as nn
import torch.optim as optim

from torchvision import models

from tqdm import tqdm

# 乱数のシードを設定
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)

In [5]:
experiment = Experiment(
    api_key="1H4oWzrtQdGj3uKZIDjQomMQs",
    project_name="phone-classification",
    workspace="yusuketakahashi2001",
)


COMET INFO: Experiment is live on comet.ml https://www.comet.ml/yusuketakahashi2001/phone-classification/0bd4956aedad46dcbd1e86cec3435c91



In [6]:
#Create Dataset
from utils.dataloader_image_classification import ImageTransform,make_datapath_list, Dataset

In [7]:
hyper_params = {
    "batch_size": 4,
    "num_epochs": 5,
}
experiment.log_parameters(hyper_params)

In [8]:
train_list = make_datapath_list(phase="train")
val_list = make_datapath_list(phase="val")

# Datasetを作成する
size = 600
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)
train_dataset = Dataset(
    file_list=train_list, transform=ImageTransform(size, mean, std), phase='train')

val_dataset = Dataset(
    file_list=val_list, transform=ImageTransform(size, mean, std), phase='val')


# DataLoaderを作成する
batch_size = hyper_params["batch_size"]

train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True)

val_dataloader = torch.utils.data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False)

# 辞書オブジェクトにまとめる
dataloaders_dict = {"train": train_dataloader, "val": val_dataloader}



./data/phone_data/train/**/*.jpg
./data/phone_data/val/**/*.jpg


In [9]:
# VGG-16モデルのインスタンスを生成
use_pretrained = True  # 学習済みのパラメータを使用
net = models.vgg16(pretrained=use_pretrained)

# VGG16の最後の出力層の出力ユニットをアリとハチの2つに付け替える
net.classifier[6] = nn.Linear(in_features=4096, out_features=2)

# 訓練モードに設定
net.train()
print('ネットワーク設定完了：学習済みの重みをロードし、訓練モードに設定しました')


Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth



ネットワーク設定完了：学習済みの重みをロードし、訓練モードに設定しました


In [10]:
# ファインチューニングで学習させるパラメータを、変数params_to_updateの1～3に格納する

params_to_update_1 = []
params_to_update_2 = []
params_to_update_3 = []

# 学習させる層のパラメータ名を指定
update_param_names_1 = ["features"]
update_param_names_2 = ["classifier.0.weight",
                        "classifier.0.bias", "classifier.3.weight", "classifier.3.bias"]
update_param_names_3 = ["classifier.6.weight", "classifier.6.bias"]

# パラメータごとに各リストに格納する
for name, param in net.named_parameters():
    if update_param_names_1[0] in name:
        param.requires_grad = True
        params_to_update_1.append(param)
        print("params_to_update_1に格納：", name)

    elif name in update_param_names_2:
        param.requires_grad = True
        params_to_update_2.append(param)
        print("params_to_update_2に格納：", name)

    elif name in update_param_names_3:
        param.requires_grad = True
        params_to_update_3.append(param)
        print("params_to_update_3に格納：", name)

    else:
        param.requires_grad = False
        print("勾配計算なし。学習しない：", name)

params_to_update_1に格納： features.0.weight
params_to_update_1に格納： features.0.bias
params_to_update_1に格納： features.2.weight
params_to_update_1に格納： features.2.bias
params_to_update_1に格納： features.5.weight
params_to_update_1に格納： features.5.bias
params_to_update_1に格納： features.7.weight
params_to_update_1に格納： features.7.bias
params_to_update_1に格納： features.10.weight
params_to_update_1に格納： features.10.bias
params_to_update_1に格納： features.12.weight
params_to_update_1に格納： features.12.bias
params_to_update_1に格納： features.14.weight
params_to_update_1に格納： features.14.bias
params_to_update_1に格納： features.17.weight
params_to_update_1に格納： features.17.bias
params_to_update_1に格納： features.19.weight
params_to_update_1に格納： features.19.bias
params_to_update_1に格納： features.21.weight
params_to_update_1に格納： features.21.bias
params_to_update_1に格納： features.24.weight
params_to_update_1に格納： features.24.bias
params_to_update_1に格納： features.26.weight
params_to_update_1に格納： features.26.bias
params_to_update_1に格納： f

In [11]:
# 損失関数の設定
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD([
    {'params': params_to_update_1, 'lr': 1e-4},
    {'params': params_to_update_2, 'lr': 5e-4},
    {'params': params_to_update_3, 'lr': 1e-3}
], momentum=0.9)


In [14]:
# モデルを学習させる関数を作成


def train_model(net, dataloaders_dict, criterion, optimizer):

    # 初期設定
    num_epochs = hyper_params["num_epochs"]
    # GPUが使えるかを確認
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print("使用デバイス：", device)

    # ネットワークをGPUへ
    net.to(device)

    # ネットワークがある程度固定であれば、高速化させる
    torch.backends.cudnn.benchmark = True

    # epochのループ
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-------------')

        # epochごとの訓練と検証のループ
        for phase in ['train', 'val']:
            if phase == 'train':
                net.train()  # モデルを訓練モードに
            else:
                net.eval()   # モデルを検証モードに

            epoch_loss = 0.0  # epochの損失和
            epoch_corrects = 0  # epochの正解数
            normal_corrects = 0
            normal_counts = 0
            broken_corrects = 0
            broken_counts = 0

            # 未学習時の検証性能を確かめるため、epoch=0の訓練は省略
            if (epoch == 0) and (phase == 'train'):
                continue

            # データローダーからミニバッチを取り出すループ
            for inputs, labels in tqdm(dataloaders_dict[phase]):

                # GPUが使えるならGPUにデータを送る
                inputs = inputs.to(device)
                labels = labels.to(device)

                # optimizerを初期化
                optimizer.zero_grad()

                # 順伝搬（forward）計算
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = net(inputs)
                    loss = criterion(outputs, labels)  # 損失を計算
                    _, preds = torch.max(outputs, 1)  # ラベルを予測

                    # 訓練時はバックプロパゲーション
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                    # 結果の計算
                    epoch_loss += loss.item() * inputs.size(0)  # lossの合計を更新
                    # 正解数の合計を更新
                    epoch_corrects += torch.sum(preds == labels.data)
                    #ラベルごとの正解数の合計を計算
                    if  preds == lebels.data:
                      if preds == lebels.data:
                        normal_corrects +=1
                      else:
                        pass
                      normal_counts +=1
                    elif labels.data == 1:
                      if preds == lebels.data:
                        broken_corrects +=1
                      else:
                        pass
                      broken_counts +=1

            

            # epochごとのlossと正解率をcomet_mlに格納
            metrics = {}
            if phase == 'train':
              metrics['train_loss'] = epoch_loss / len(dataloaders_dict[phase].dataset)
              metrics['train_acc']  = epoch_corrects.double() / len(dataloaders_dict[phase].dataset)
              metrics['train_normal_acc'] = normal_corrects.double() / normal_counts
              metrics['train_broken_acc'] = broken_corrects.double() / broken_counts


            elif phase == 'val':
              metrics['test_loss']  =  epoch_loss / len(dataloaders_dict[phase].dataset)
              metrics['test_acc']  = epoch_acc = epoch_corrects.double() / len(dataloaders_dict[phase].dataset)
              metrics['train_normal_acc'] = normal_corrects.double() / normal_counts
              metrics['train_broken_acc'] = broken_corrects.double() / broken_counts




            experiment.log_metrics(metrics, step=epoch)
            print('Epoch:{}'.format(epoch))
            
    experiment.end()
    # PyTorchのネットワークパラメータの保存
    save_path = './saveweight/weights_fine_tuning.pth'
    torch.save(net.state_dict(), save_path)
    print('Finish')


In [ ]:
# 学習・検証を実行する
train_model(net, dataloaders_dict, criterion, optimizer)

In [25]:
val_dataloader

In [36]:
train_ = next(iter(dataloaders_dict['val']))
print(train_)

[tensor([[[[2.0434, 2.0263, 1.9920,  ..., 2.0434, 2.0605, 2.0605],
          [2.0263, 2.0263, 1.9920,  ..., 2.0777, 2.0777, 2.0605],
          [2.0434, 2.0434, 2.0263,  ..., 2.0948, 2.0777, 2.0605],
          ...,
          [2.2489, 2.2489, 2.2489,  ..., 2.2489, 2.2489, 2.2489],
          [2.2489, 2.2489, 2.2489,  ..., 2.2489, 2.2489, 2.2489],
          [2.2489, 2.2489, 2.2489,  ..., 2.2489, 2.2489, 2.2489]],

         [[2.2010, 2.1835, 2.1485,  ..., 2.1310, 2.1485, 2.1660],
          [2.1835, 2.1835, 2.1485,  ..., 2.1660, 2.1660, 2.1660],
          [2.2010, 2.2010, 2.1835,  ..., 2.1835, 2.1660, 2.1485],
          ...,
          [2.4286, 2.4286, 2.4286,  ..., 2.4286, 2.4286, 2.4286],
          [2.4286, 2.4286, 2.4286,  ..., 2.4286, 2.4286, 2.4286],
          [2.4286, 2.4286, 2.4286,  ..., 2.4286, 2.4286, 2.4286]],

         [[2.0997, 2.0823, 2.0474,  ..., 1.9951, 2.0125, 2.0125],
          [2.0823, 2.0823, 2.0474,  ..., 2.0474, 2.0300, 2.0300],
          [2.0997, 2.0997, 2.0823,  ..., 

In [27]:
for (data1, data2), label in loader:
        print(data1.shape)
        print(data2.shape)
        print(label)

ValueError: ignored